In [ ]:
import nltk
from wikidata.client import Client
from wikidata.entity import Entity
from collections import defaultdict, deque
import requests, json
nltk.download('punkt')

MAX_LEVEL = 2

In [ ]:
text = "Home of Mark Zuckerberg is great! House"
def get_nouns(sentence):
    tokens = nltk.word_tokenize(sentence)
    tags = nltk.pos_tag(tokens)
    retVal = []
    i = 0
    while i < len(tags):
        key, tag = tags[i]
        if 'NN' in tag:
            current_noun = key
            j = i + 1
            while j < len(tags):
                key_j, tag_j = tags[j]
                if 'NN' in tag_j:
                    current_noun = ' '.join([current_noun, key_j])
                    j += 1
                else:
                    i = j - 1
                    break
            retVal.append(current_noun)
            if j == len(tags):
                return retVal          
        i += 1
    return retVal
    
print(get_nouns(text))

In [ ]:
def get_wikidata_id(item):
    try:
        response = requests.get(f'https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&titles={item}&format=json')
        wikidata_id = list(json.loads(response.text)['query']['pages'].values())[0]['pageprops']['wikibase_item']
        return wikidata_id
    except:
        return None


In [ ]:
def bfs(id):
    seen = set()
    q = deque([(id, 0)])
    client = Client()
    retDict = defaultdict(list)
    while q:
        ent, level = q.popleft()
        if ent in seen or level > MAX_LEVEL:
            continue
        retDict[level].append(ent)
        seen.add(ent)
        try:
            e = client.get(ent if isinstance(ent, str) else ent.id).values()
            entities = []
            for ent in list(e)[:min(len(e), 20)]:
                if isinstance(ent, Entity):
                    entities.append(ent)
            for entity in entities:
                q.append((entity, level + 1))
        except:
            continue
    return retDict

In [ ]:
id = get_wikidata_id('Mark Zuckerberg')
x = bfs(id)
x